In [ ]:
!pip install fsspec
!pip install gcsfs
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 12.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 16.5 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |████████████████████████████████| 271 kB 66.5 MB/s 
     |████████████████████████████████| 144 kB 62.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 12.0 MB/s 
     |████████████████████████████████| 596 kB 58.4 MB/s 
     |████████████████████████████████| 6.6 MB 35.6 MB/s 
     |████████████████████████████████| 101 kB 11.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.o

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ln -s "/content/drive/My Drive/Colab Notebooks/tfm"
import sys
sys.path.append('tfm/checkpoints/')
sys.path.append('tfm/outputs/')
sys.path.append('tfm/checkpoints/beto_vola_paragraph')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ln: failed to create symbolic link './tfm': File exists


In [ ]:
from datasets import Dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import train_test_split, KFold
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tqdm.auto import tqdm
from transformers import (
    AutoTokenizer,
    TFBertForSequenceClassification,
)

In [ ]:
# filepath = "../../data/outputs/dataset_2021_paragraph"
# filepath = "https://storage.googleapis.com/tfm_aideas_datasets/dataset_2021_paragraph" # Filepath GCloud
# filepath = "tfm/outputs/dataset_2021_paragraph" # Filepath COLAB

filepath = "https://storage.googleapis.com/tfm_aideas_datasets/dataset_news_total_no"

In [ ]:
labeled_df = pd.read_csv(f"{filepath}_filters.csv", sep=";", index_col=0, parse_dates=["date"], low_memory=False, lineterminator="\n")
labeled_df.shape

(241553, 42)

In [ ]:
labeled_df.sort_values(by=["date"], inplace=True)

In [ ]:
labeled_df = labeled_df[(labeled_df.finanzas_tag_s_proba>=0.045)&(labeled_df.impacto_tag_s_proba>=0.15)]

In [ ]:
labeled_df.vola_label = labeled_df.vola_label + 1

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
labeled_df.vola_label.value_counts()/labeled_df.vola_label.count()

2.0    0.398258
0.0    0.305457
1.0    0.296285
Name: vola_label, dtype: float64

# Transfer Learning con BETO

In [ ]:
N_LABELS = len(np.unique(labeled_df.vola_label))

In [ ]:
checkpoint = "dccuchile/bert-base-spanish-wwm-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/310 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/650 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [ ]:
model = TFBertForSequenceClassification.from_pretrained(checkpoint, num_labels=N_LABELS)

Downloading:   0%|          | 0.00/512M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.layers[0].trainable=False

In [ ]:
model.compile(
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
optimizer=Adam(learning_rate=0.0001),
metrics=['accuracy']
)

In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109850880 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 109,853,187
Trainable params: 2,307
Non-trainable params: 109,850,880
_________________________________________________________________


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(labeled_df.body_no_entities, labeled_df.vola_label, test_size=0.2, random_state=42)

In [ ]:
x_train = tokenizer(x_train.tolist(), padding="max_length", truncation=True, return_tensors="tf")

In [ ]:
x_test = tokenizer(x_test.tolist(), padding="max_length", truncation=True, return_tensors="tf")

In [ ]:
N_EPOCAS = 150

In [ ]:
# filepath_checkpoint = './checkpoints/finanzas_tag_s_n/checkpoint'
import os

filepath_checkpoint = 'tfm/checkpoints/beto_vola_paragraph/checkpoint' # Filepath COLAB

if not os.path.exists(filepath_checkpoint):
  os.mkdir('tfm/checkpoints/beto_vola_paragraph')

sys.path.append('tfm/checkpoints/beto_vola_paragraph')

In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    filepath=filepath_checkpoint,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

early_stopping = EarlyStopping(monitor='val_loss', patience=15, mode='min')

hist = model.fit(
    dict(x_train),
    y_train,
    batch_size=128,
    epochs=N_EPOCAS,
    validation_split=0.1,
    callbacks=[model_checkpoint_callback, early_stopping]
    )

Epoch 1/150
879/879 [==============================] - 3178s 4s/step - loss: 1.0900 - accuracy: 0.3960 - val_loss: 1.0872 - val_accuracy: 0.3957
Epoch 2/150
879/879 [==============================] - 3155s 4s/step - loss: 1.0879 - accuracy: 0.3988 - val_loss: 1.0877 - val_accuracy: 0.3967
Epoch 3/150
879/879 [==============================] - 3155s 4s/step - loss: 1.0870 - accuracy: 0.4005 - val_loss: 1.0858 - val_accuracy: 0.4028
Epoch 4/150
879/879 [==============================] - 3155s 4s/step - loss: 1.0860 - accuracy: 0.4017 - val_loss: 1.0857 - val_accuracy: 0.4043
Epoch 5/150
879/879 [==============================] - 3155s 4s/step - loss: 1.0859 - accuracy: 0.4024 - val_loss: 1.0856 - val_accuracy: 0.4054
Epoch 6/150
879/879 [==============================] - 3155s 4s/step - loss: 1.0856 - accuracy: 0.4034 - val_loss: 1.0857 - val_accuracy: 0.4071
Epoch 7/150
879/879 [==============================] - 3153s 4s/step - loss: 1.0848 - accuracy: 0.4035 - val_loss: 1.0856 - val_ac

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.title("Train - Validation Loss")
plt.plot(hist.history['loss'], label='train')
plt.plot(hist.history['val_loss'], label='validation')
plt.xlabel('num_epochs', fontsize=12)
plt.ylabel('loss', fontsize=12)
plt.legend(loc='best')
plt.grid()
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.title("Train - Validation Accuracy")
plt.plot(hist.history['accuracy'], label='train')
plt.plot(hist.history['val_accuracy'], label='validation')
plt.xlabel('num_epochs', fontsize=12)
plt.ylabel('accuracy', fontsize=12)
plt.legend(loc='best')
plt.grid()
plt.show()

In [ ]:
model.load_weights(filepath_checkpoint)
model.evaluate(x_test.data, y_test)

In [ ]:
# Comparativa con una predicción por clase mayoritaria de train:

pd.Series(y_train).value_counts()/sum(pd.Series(y_train).value_counts())

In [ ]:
y_predictions_train = model.predict(x_train.data)

In [ ]:
train_posprob = tf.nn.softmax(y_predictions_train.logits).numpy()[:,0]

In [ ]:
train_fpr, train_tpr, train_thr = roc_curve(y_train, train_posprob, pos_label=0)

In [ ]:
y_predictions_test = model.predict(x_test.data)

In [ ]:
test_posprob = tf.nn.softmax(y_predictions_test.logits).numpy()[:,0]

In [ ]:
test_fpr, test_tpr, test_thr = roc_curve(y_test, test_posprob, pos_label=0)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
plt.plot(train_fpr, train_tpr, color='r', label='train')
plt.plot(test_fpr, test_tpr, color='g', label='test')

plt.plot([0,1],[0,1])
plt.xlim([0.0, 1.0])
plt.ylim([0.0,1.0])
plt.legend(loc='lower right')
plt.show()

In [ ]:
y_pred = y_predictions_test.logits.argmax(axis=1)

In [ ]:
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
def plot_cm(labels, predictions, p=0.5):
    cm = confusion_matrix(labels, predictions)
    plt.figure(figsize=(5, 5))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title("Confusion matrix (non-normalized))")
    plt.ylabel("Actual label")
    plt.xlabel("Predicted label")


plot_cm(y_test, y_pred)

In [ ]:
def encode(input):
    return tokenizer(input["body_no_entities"], padding="max_length", truncation=True)

In [ ]:
dataset_opt = Dataset.from_pandas(labeled_df.loc[-10000:,["body_no_entities","vola_label"]])

In [ ]:
tokenized_dataset = dataset_opt.map(encode, batched=True)
tf_tokenized_dataset = tokenized_dataset.remove_columns(["pk", "body_no_entities",  "vola_label"])
tf_tokenized_dataset.set_format(type='tensorflow', columns=['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
results = model.predict(tf_tokenized_dataset.to_tf_dataset(batch_size=256), verbose=1)

In [ ]:
labeled_df.loc[-10000:, "vola_label"] = np.NaN
labeled_df.loc[-10000:, "vola_label"] = results.logits.argmax(axis=1)

In [ ]:
# def func_apply(row):
#     if (not type(row.body_no_entities)==str):    
#         row.body_no_entities = str(row.body_no_entities)
#     x = tokenizer(row.body_no_entities, padding="max_length", truncation=True, return_tensors="tf")
#     y_pred = model.predict(x.data)
#     row['finanzas_tag_s_n'] = int(y_pred.logits.argmax(axis=1)[0])    
#     row['finanzas_tag_s_n_proba'] = tf.nn.softmax(y_pred.logits).numpy().max(axis=1)[0]
#     return row

In [ ]:
# tqdm.pandas()
# df_total_labels["finanzas_tag_s_n_proba"] = np.NaN
# df_total_labels.loc[finance_label, "finanzas_tag_s_n_proba"] = 1.
# df_total_labels.loc[~finance_label] = df_total_labels.loc[~finance_label].progress_apply(lambda row : func_apply(row), axis=1)

In [ ]:
labeled_df.to_csv(f'{filepath}_predict.csv', sep=";")